<a href="https://colab.research.google.com/github/magiwanders/CMLS_HW1/blob/master/1_FeatureAnalysisAndComputation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FEATURE ANALYSIS FOR THE DATASET**
### Extract, plot and compare different features for the dataset (e.g. MFCC). Choose the features that enable the best differentiation between the classes.

# Mount the drive and enter the dataset directory

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/CMLS_HW1_UrbanClassification/dataset"
%ls | grep fold

Mounted at /content/drive
[Errno 2] No such file or directory: '/content/drive/MyDrive/CMLS_HW1_UrbanClassification/dataset'
/content


# Import libraries

In [ ]:
import numpy as np
import librosa
import os
import matplotlib.pyplot as plt
import sklearn.svm
import IPython.display as ipd
import scipy as sp
from pathlib import Path
import pandas as pd
import re
import pickle

#Import the Dataset Metadata

In [ ]:
metadata = pd.read_csv('UrbanSound8K.csv')
metadata.head()

# Define helper functions

In [ ]:
def compute_mfcc(audio, fs, win_length, hop_size, n_mfcc):
    # Compute the spectrogram of the audio signal
    X = np.abs(librosa.stft(
        audio,
        window='hamming',
        n_fft=win_length,
        hop_length=hop_size,)
        )
    
    # Find the weights of the mel filters
    mel = librosa.filters.mel(
        sr=fs,
        n_fft=win_length,
        n_mels=40,
        fmin=0,
        fmax=fs,
    )
    
    # Apply the filters to spectrogram
    melspectrogram = np.dot(mel, X)
    # Take the logarithm
    log_melspectrogram = np.log10(melspectrogram + 1e-16)
    
    # Apply the DCT to log melspectrogram to obtain the coefficients
    mfcc = sp.fftpack.dct(log_melspectrogram, axis=0, norm='ortho')[1:n_mfcc+1]
    return mfcc

In [ ]:
def extract_features(x, fs, win_length, hop_size, n_mfcc):
    mfcc = compute_mfcc(x, fs, win_length, hop_size, n_mfcc);

    # take the statistics over time of the mfccs
    min = np.min(mfcc, axis=1);
    max = np.max(mfcc, axis=1);
    mean = np.mean(mfcc, axis=1);
    median = np.median(mfcc, axis=1);
    variance = np.var(mfcc, axis=1);

    # in total I should have 25*5 = 125 features per audio frame
    features = np.empty((0,125))
    ext_features = np.hstack([min, max, mean, median, variance])
    features = np.vstack([features,ext_features])

    return features

# Compute the features for each dataset fold

In [ ]:
# Assume that the dataset is in the current directory
dataset_path = Path(".")

Fs = 22050;

win_length = int(np.ceil(0.0232*Fs))   # should return a 512 samples window
hop_size = int(0.5*win_length)

n_mfcc = 25;

mfcc_data = []

# For each directory, which corresponds to a fold ...
for current_fold_dir in dataset_path.iterdir():
  # Check if the directory is really a directory
  if current_fold_dir.is_dir():
    # Save the current fold number
    current_fold_number = re.findall('[0-9-]+', str(current_fold_dir)) # Extract the fold number with regex
    print("Scanning fold {} of 10" .format(current_fold_number))#, end='\x1b[1K\r') # Status printing with line clearing

    # For each audio file in current_fold_dir
    for current_audio_dir in (current_fold_dir).iterdir():
      # Check if it's really a file and not a fold
      if not current_audio_dir.is_dir() and os.path.splitext(current_audio_dir)[1] == '.wav' and current_fold_number[0] == '1' :
        filename = current_audio_dir.stem + '.wav'
        print("Currently processing: {}" .format(filename))
      
        x, sr = librosa.load(current_audio_dir, sr=Fs)

        features = extract_features(x, Fs, win_length, hop_size, n_mfcc)

        metadata_row = metadata.loc[metadata['slice_file_name']==filename].values.tolist()
        label = metadata_row[0][-1];
        label_id = metadata_row[0][-2];
        fold = metadata_row[0][-3]
        
        mfcc_data.append([features, features.shape, label_id, label, fold])

#///////////////////////////////////////////////////////////////////////////////
#///////////////////// PARALLELIZED EQUIVALENT /////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////

# def analyze_folder(current_fold_dir):
#   # Check if the directory is really a directory
#   if current_fold_dir.is_dir():
#     # Save the current fold number
#     current_fold_number = re.findall('[0-9-]+', str(current_fold_dir)) # Extract the fold number with regex
#     print("Scanning fold {} of 10" .format(current_fold_number))#, end='\x1b[1K\r') # Status printing with line clearing

#     # For each audio file in current_fold_dir
#     for current_audio_dir in (current_fold_dir).iterdir():
#       # Check if it's really a file and not a fold
#       if not current_audio_dir.is_dir() and os.path.splitext(current_audio_dir)[1] == '.wav' and current_fold_number[0] == '1' :
#         filename = current_audio_dir.stem + '.wav'
#         print("Currently processing: {}" .format(filename))
      
#         x, sr = librosa.load(current_audio_dir, sr=Fs)

#         features = extract_features(x, Fs, win_length, hop_size, n_mfcc)

#         metadata_row = metadata.loc[metadata['slice_file_name']==filename].values.tolist()
#         label = metadata_row[0][-1];
#         label_id = metadata_row[0][-2];
#         fold = metadata_row[0][-3]
        
#         mfcc_data.append([features, features.shape, label_id, label, fold])

# if __name__ == '__main__':
#     out_parall = p_map(analyze_folder, dataset_path.iterdir())


In [ ]:
cols=["features", "shape","label_id", "label", "fold"]
mfcc_pd = pd.DataFrame(data = mfcc_data, columns=cols)

In [ ]:
mfcc_pd.head()

In [ ]:
labels = set(mfcc_pd['label'])
print(labels)
cnt = [[label,list(mfcc_pd['label']).count(label)] for label in labels]
dict_cnt = dict(cnt)
dict_cnt

In [ ]:
ll = [mfcc_pd['features'][i].ravel() for i in range(mfcc_pd.shape[0])]
mfcc_pd['sample'] = pd.Series(ll, index=mfcc_pd.index)
del mfcc_pd['features']

In [ ]:
mfcc_pd.head()